In [ ]:
pip install pandas_market_calendars

In [40]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
from datetime import timedelta

from yahoo_fin.stock_info import get_data, get_balance_sheet
from get_all_tickers import get_tickers as gt # want to use this for tickers data

import pandas_market_calendars as mcal

from progressbar import ProgressBar


In [2]:
def pull_yahoo_data(ticker, start, end, dur):
    '''
    Pulls adjusted close, date, and ticker date for the given stock, start, and end date.
    Depending on duration, this can pull daily, weekly, or monthly data
    '''
    
    yahoo = get_data(ticker, start_date=start, end_date=end, index_as_date=False, interval=dur)[["date", "adjclose", "ticker"]]
    return yahoo

In [3]:
start_date = dt.datetime.strptime("2000-01-01", "%Y-%m-%d")
end_date = dt.datetime.now()

df = pull_yahoo_data('AMZN', start_date, end_date, '1d')

In [4]:
df.head()

,date,adjclose,ticker
0,2000-01-03,89.3750,AMZN
1,2000-01-04,81.9375,AMZN
2,2000-01-05,69.7500,AMZN
3,2000-01-06,65.5625,AMZN
4,2000-01-07,69.5625,AMZN


In [5]:
# Obtain stock tickers

data = pd.read_html('https://en.wikipedia.org/wiki/List_of_S&P_500_companies')
#print(data)
table = data[0]
#table.head()
tickers = table['Symbol'].tolist()

In [6]:
allData = pd.DataFrame(columns=['ticker', 'date', 'twoWeeksClose', 'threeMonthsClose', 'oneYearClose'])
investing = pd.read_csv('newData/investingReddit.csv')

In [7]:
investing = investing[['Date', 'Ticker']]
investing.head()

,Date,Ticker
0,1.604213e+09,MMM
1,1.614315e+09,MMM
2,1.605132e+09,MMM
3,1.598891e+09,MMM
4,1.596129e+09,MMM


In [8]:
investing['Date'] = investing["Date"].apply(dt.datetime.fromtimestamp)
investing.head()

,Date,Ticker
0,2020-11-01 01:47:57,MMM
1,2021-02-25 23:56:36,MMM
2,2020-11-11 17:06:19,MMM
3,2020-08-31 12:24:44,MMM
4,2020-07-30 13:17:10,MMM


In [ ]:
pbar = ProgressBar()

for row in pbar(investing.iterrows()):
    ticker = row[1][1]
    date = row[1][0]
#     print(date + timedelta(days=14))
#     print((date.date() + timedelta(days=14)))
#     print((date + timedelta(days=14)).date())
    
    
    try:
        twoWeeks = pull_yahoo_data(ticker, (date + timedelta(days=14)).date(), (date + timedelta(days=16)).date(), '1d')
    except:
        twoWeeks = {}
        twoWeeks['adjclose'] = [None]
    try:
        threeMonths = pull_yahoo_data(ticker, (date + timedelta(days=90)).date(), (date + timedelta(days=93)).date(), '1d')
    except:
        threeMonths = {}
        threeMonths['adjclose'] = [None]
    try:
        oneYear = pull_yahoo_data(ticker, (date + timedelta(days=365)).date(), (date + timedelta(days=367)).date(), '1d')
    except:
        oneYear = {}
        oneYear['adjclose'] = [None]
    
    tempdict = dict()
    tempdict['ticker'] = ticker
    tempdict['date'] = date
    tempdict['twoWeeksClose'] = twoWeeks['adjclose'][0]
    tempdict['threeMonthsClose'] = threeMonths['adjclose'][0]
    tempdict['oneYearClose'] = oneYear['adjclose'][0]
    tempdict['5dp'] = previousClose['adjclose'].values[0]
    tempdict['4dp'] = previousClose['adjclose'].values[1]
    tempdict['3dp'] = previousClose['adjclose'].values[2]
    tempdict['2dp'] = previousClose['adjclose'].values[3]
    tempdict['1dp'] = previousClose['adjclose'].values[4]

    allData = allData.append(pd.DataFrame(pd.Series(tempdict)).T)



In [ ]:
allData

In [ ]:
pbar = ProgressBar()

for i, row in pbar(enumerate(investing.head().iterrows())):
    ticker = row[1][1]
    date = row[1][0]
    
    
    try:
        previousCloses = pull_yahoo_data(ticker, (date - timedelta(days=10)).date(), date.date(), '1d')
        previousCloses = previousCloses.tail()
        print(previousCloses)
    except:
        previousCloses = {}
        previousCloses['adjclose'] = [None]
        
    allData['5dp'] = previousCloses['adjclose'][-1]


In [9]:
early = nyse.schedule(start_date='2012-07-01', end_date='2012-07-10')



# Create a calendar
nyse = mcal.get_calendar('NYSE')
mcal.date_range(early, frequency='1D')


NameError: name 'nyse' is not defined

In [10]:
dailyClose = pd.read_csv('yahoo_daily_sp500.csv')

In [11]:
dailyClose.head(20)

,Unnamed: 0,date,ticker,adjclose
0,0,2000-01-03,CB,10.392402
1,1,2000-01-03,EFX,10.855853
2,2,2000-01-03,J,7.692913
3,3,2000-01-03,CI,23.833805
4,4,2000-01-03,COP,9.057103
5,5,2000-01-03,FDX,37.847946
6,6,2000-01-03,IP,27.415386
7,7,2000-01-03,RSG,6.323537
8,8,2000-01-03,CAT,13.755004
9,9,2000-01-03,LEN,5.994212


In [12]:
twitterTickers = pd.read_csv('twitter_tickers.csv')
# twitterTickers = twitterTickers[1:]

In [13]:
len(twitterTickers)

25283

In [14]:
redditTickers = pd.read_csv('reddit_tickers.csv')

In [15]:
len(redditTickers)

506

In [16]:
dailyClose['ticker'].nunique()

499

In [17]:
twitterTickers = twitterTickers['0']

In [17]:
twitterTickers.head(10)

0       T
1      HD
2     BAC
3    GOOG
4      GE
5      FB
6     DIS
7     DHR
8       D
9      GD
Name: 0, dtype: object

In [18]:
dailyCloseTickers = dailyClose['ticker'].unique()

In [19]:
rogueTickers = []

pbar = ProgressBar()

for ticker in pbar(twitterTickers):
    if ticker not in dailyCloseTickers:
        rogueTickers.append(ticker)

100% |########################################################################|


In [20]:
len(rogueTickers)

24784

In [21]:
rogueTickers

['UTX',
 'NGG',
 'BP',
 'TSLA',
 'UN',
 'BABA',
 'CELG',
 'TM',
 'TOT',
 'BHP',
 'PCLN',
 'NVS',
 'IEP',
 'TSM',
 'BUD',
 'UL',
 'AUDVF',
 'HP',
 'ESYJY',
 'SAP',
 'HNNMY',
 'HYMTF',
 'AABA',
 'TRI',
 '005930',
 'TRIP',
 'ADDYY',
 'NSRGY',
 'GILLETTE',
 'NSANY',
 'ASC.L',
 'HMC',
 'CBS',
 'SNE',
 'DB',
 'CHL',
 'HSBC',
 'VWAGY',
 'RDS.A',
 'BURBY',
 'RYAAY',
 'SNY',
 'REX',
 'TSCDD',
 'CODI',
 'PCG',
 'BBL',
 'SIEGY',
 'NEXT',
 'BMWYY',
 'LRLCY',
 'DANOY',
 'GRPN',
 'THLLY',
 'EQNR',
 'ALIZY',
 'BAYRY',
 'SNP',
 'AZN',
 'SAN',
 'HEINY',
 'ABB',
 'PTR',
 'BSAC',
 'PICO',
 'HRG',
 'BRK-A',
 'TFCF',
 'GSK',
 'AGFS',
 'BASFY',
 'SPLP',
 'RDS-B',
 'BCH',
 'VOD',
 'CRRFY',
 'FLR',
 'JEC',
 'LUK',
 'ARRY',
 'HRS',
 'DCIX',
 'SRCL',
 'SLM',
 'NVAX',
 'HIBB',
 'ODP',
 'GNTX',
 'WYN',
 'AMG',
 'PAH',
 'FMSA',
 'CX',
 'DLPH',
 'DDR',
 'NUAN',
 'GNCA',
 'FNSR',
 'MUR',
 'HDS',
 'TMK',
 'TRQ',
 'PDCO',
 'FLEX',
 'ETE',
 'XL',
 'PTEN',
 'HCP',
 'MON',
 'COL',
 'SLCA',
 'LITE',
 'CY',
 'EQT',
 'DPS',

In [ ]:
badTickers = []

pbar = ProgressBar()

for ticker in pbar(rogueTickers):
    if not valid_ticker(ticker):
        badTickers.append(ticker)

In [21]:
badTickers = []

In [26]:
badTickers

['UTX',
 'CELG',
 'PCLN',
 'AABA',
 '005930',
 'GILLETTE',
 'CBS',
 'CHL',
 'VWAGY',
 'RDS.A',
 'TSCDD',
 'PICO',
 'HRG',
 'TFCF',
 'JEC',
 'LUK',
 'HRS',
 'DCIX',
 'WYN',
 'PAH',
 'FMSA',
 'DLPH',
 'DDR',
 'FNSR',
 'HDS',
 'TMK',
 'ETE',
 'HCP',
 'MON',
 'COL',
 'CY',
 'DPS',
 'EVHC',
 'NBL',
 'AET',
 'VIAB',
 'ticker',
 'HTZ',
 'MNK',
 'SYMC',
 'NFX',
 'APC',
 'LLL',
 'STI',
 'ESRX',
 'BHGE',
 'CRZO',
 'RHT',
 'AMTD',
 'TWX',
 'MYL',
 'ETFC',
 'BBT',
 'AGN',
 'GG',
 'VRX',
 'GGP',
 'CTL',
 'ESV',
 'SQQQ',
 'TVIX',
 'SPN',
 'RTN',
 'TSS',
 'TQQQ',
 'WPX',
 'WFT',
 'AMZN-GPS',
 'BTC',
 'INTC-USB',
 'FB-GOOGL-GOOG',
 'ABX',
 'QQQ',
 'S',
 'OMG',
 'AVEO',
 'SIL',
 'CBL',
 'CDE',
 'IMMU',
 'UEC',
 'FIT',
 'DRIV',
 'PDLI',
 'FPL',
 'HLX',
 'HTGC',
 'ARB',
 'EGOV',
 'JOE',
 'AIMT',
 'AWAY',
 'FRAN',
 'AONE',
 'ASIX',
 'ARCB',
 'CTR',
 'USPH',
 'TRUP',
 'PE',
 'GNOM',
 'HERO',
 'AIQ',
 'GMO',
 'JCOM',
 'CEM',
 'VSLR',
 'PRIM',
 'PRS',
 'ACT',
 'PBY',
 'RA',
 'WINS',
 'CHIC',
 'CHEF',
 'ACV',

In [33]:
len(badTickers)

13687

In [34]:
pbar = ProgressBar()
for ticker in pbar(range(20000, len(rogueTickers))):
#     print(ticker)
    try:
        get_balance_sheet(rogueTickers[ticker])
    except Exception as e:
#         print(f'On ticker {ticker} there was this error: {e}')
        badTickers.append(rogueTickers[ticker])
        print(ticker)

20000
20001
20006
20008
20009
20010
20011
20013
20014
20015
20016
20017
20018
20019
20020
20021
20022
20023
20024
20025
20027
20028
20029
20030
20031
20036
20037
20039
20041
20044
20045


20047
20048
20049
20051
20053
20054
20055
20056
20057
20058
20059
20060
20061
20062
20063
20064
20066
20067
20069
20070
20071
20074
20077
20084
20085
20086
20088
20089
20090
20091
20092
20093
20094


20095
20096
20097
20098
20099
20100
20101
20102
20103
20104
20105
20106
20107
20108
20109
20110
20111
20112
20113
20115
20116
20117
20118
20119
20120
20121
20122
20123
20124
20125
20126
20127
20128
20129
20130
20131
20132
20133
20134
20135
20136
20137
20138
20139
20140
20141
20142


20143
20144
20145
20148
20149
20150
20151
20152
20153
20155
20156
20157
20159
20160
20161
20162
20163
20165
20166
20168
20169
20170
20171
20172
20173
20174
20176
20180
20182
20183
20184
20185
20186
20187
20188
20190


20191
20192
20193
20194
20195
20196
20197
20198
20199
20201
20202
20203
20206
20207
20208
20209
20210
20211
20212
20213
20215
20216
20218
20220
20222
20224
20225
20226
20227
20228
20229
20230
20231
20232
20233
20235
20236
20237
20238


...

24607
24608
24609
24610
24611
24612
24614
24615
24616
24618
24619
24620
24622
24623
24626
24627
24628
24629
24630
24632
24633
24634
24635
24636
24637
24639
24640
24642
24645
24646
24647
24648
24649
24650
24651
24652
24653
24654


24655
24656
24657
24658
24659
24660
24662
24663
24664
24666
24667
24668
24669
24670
24671
24672
24673
24674
24675
24676
24678
24679
24680
24681
24682
24683
24684
24685
24686
24687
24688
24690
24691
24692
24694
24697
24698
24699
24700
24701
24702


24703
24704
24705
24706
24707
24708
24709
24710
24711
24712
24713
24714
24715
24716
24717
24718
24719
24720
24721
24727
24728
24730
24732
24736
24737
24740
24742
24745


24752
24759
24763
24765
24766
24779
24782


24783


In [36]:
pd.Series(badTickers).to_csv('badTickers.csv')

In [37]:
goodTickers = []
pbar = ProgressBar()


for ticker in pbar(rogueTickers):
    if ticker not in badTickers:
        goodTickers.append(ticker)

100% |########################################################################|


In [38]:
goodTickers500 = []
pbar = ProgressBar()
for ticker in pbar(twitterTickers):
    if ticker not in badTickers:
        goodTickers500.append(ticker)

100% |########################################################################|


In [39]:
pd.Series(goodTickers).to_csv('goodTickers.csv')
pd.Series(goodTickers500).to_csv('goodTickers500.csv')

In [41]:
allTheTwitter = pd.read_csv('allthetwitter.csv')[['ticker', 'date', 'text']]

C:\Users\Haig.Bernard\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (0,1,2,3) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [43]:
dailyClose = pd.read_csv('yahoo_daily_sp500.csv')
uniqueTickers = dailyClose['ticker'].unique()
len(uniqueTickers)

499

In [47]:
sp500 = allTheTwitter.loc[allTheTwitter['ticker'].isin(uniqueTickers)]

In [52]:
sp500['date'] = pd.to_datetime(sp500['date'])

<ipython-input-52-fe6ceb1c7edd>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sp500['date'] = pd.to_datetime(sp500['date'])


In [55]:
min(sp500['date'])

Timestamp('2008-10-03 00:00:00')

In [58]:
dailyClose = dailyClose.loc[dailyClose['date'] > '2008-09-01'][['date', 'ticker', 'adjclose']]
dailyClose.head(10)

,date,ticker,adjclose
891215,2008-09-02,MTD,105.870003
891216,2008-09-02,WFC,22.260929
891217,2008-09-02,EQR,23.168758
891218,2008-09-02,SNPS,21.190001
891219,2008-09-02,UAA,4.403750
891220,2008-09-02,MCD,43.610111
891221,2008-09-02,UNP,31.635942
891222,2008-09-02,AAPL,5.112499
891223,2008-09-02,CLX,41.927345
891224,2008-09-02,SHW,51.617455


In [77]:
dailyClose['date'] = pd.to_datetime(dailyClose['date'])

In [89]:
allData = pd.DataFrame(columns=['text', 'ticker', 'date', 'twoWeeksClose', 'threeMonthsClose', 'oneYearClose'])

In [ ]:
pbar = ProgressBar()

for uniqTicker in pbar(uniqueTickers):
    tempData = sp500.loc[sp500['ticker'] == uniqTicker].sort_values('date', ascending=True)
    tempClose = dailyClose.loc[dailyClose['ticker'] == uniqTicker].set_index('date')
    
    for row in tempData.iterrows():
        tempdict = dict()
        ticker = row[1][0]
        date = row[1][1]
        text = row[1][2]
        tempdict['ticker'] = ticker
        tempdict['date'] = date
        tempdict['text'] = text
        
        try:
            tempTimeframe = tempClose.loc[date - timedelta(days=10) : date].tail(6)
            tempdict['5dp'] = tempTimeframe['adjclose'].iloc[0]
            tempdict['4dp'] = tempTimeframe['adjclose'].iloc[1]
            tempdict['3dp'] = tempTimeframe['adjclose'].iloc[2]
            tempdict['2dp'] = tempTimeframe['adjclose'].iloc[3]
            tempdict['1dp'] = tempTimeframe['adjclose'].iloc[4]
            tempdict['dayOf'] = tempTimeframe['adjclose'].iloc[5]
        except Exception as e:
            tempdict['5dp'] = None
            tempdict['4dp'] = None
            tempdict['3dp'] = None
            tempdict['2dp'] = None
            tempdict['1dp'] = None
            tempdict['dayOf'] = None

        try:
            twoWeeks = tempClose.loc[date + timedelta(days=14): date + timedelta(days=18)].head(1)
            tempdict['twoWeeksClose'] = twoWeeks['adjclose'].iloc[0]
        except:
            tempdict['twoWeeksClose'] = None

        try:
            threeMonths = tempClose.loc[date + timedelta(days=90): date + timedelta(days=93)].head(1)
            tempdict['threeMonthsClose'] = threeMonths['adjclose'][0]
        except:
            tempdict['threeMonthsClose'] = None

        try:
            oneYear = tempClose.loc[date + timedelta(days=365): date + timedelta(days=368)].head(1)
            tempdict['oneYearClose'] = oneYear['adjclose'][0]
        except:
            tempdict['oneYearClose'] = None



        allData = allData.append(pd.DataFrame(pd.Series(tempdict)).T)
    

In [88]:
allData

,text,ticker,date,twoWeeksClose,threeMonthsClose,oneYearClose,5dp,4dp,3dp,2dp,1dp,dayOf
0,GLOBAL MARKETS Stocks rally on Wells Wachovia ...,C,2008-10-03,128.95,62.6519,41.0758,174.62,153.822,177.74,199.319,194.986,159.022
0,GLOBAL MARKETS Credit crisis fears spark globa...,C,2008-10-06,130.77,62.1254,41.0758,153.822,177.74,199.319,194.986,159.022,150.876
0,GLOBAL MARKETS U S stocks surge on bargain hu...,C,2008-10-16,115.037,39.7497,40.3721,112.052,122.278,136.49,161.361,140.65,137.79
0,UPDATE 1 POLL S Korea Oct export growth seen a...,C,2008-10-31,83.5359,34.3031,35.0947,105.206,101.652,116.211,111.878,115.037,119.776
0,FOREX Easing risk aversion cools dollar yen r...,C,2008-11-03,78.0077,32.1042,35.5345,101.652,116.211,111.878,115.037,119.776,122.759


In [ ]:
pbar = ProgressBar()


for i in pbar(range(0, len(sp500))):
    
    tempdict = dict()
    tweet = sp500['text']
    ticker = sp500['ticker'][i]
    date = sp500['date'][i]
    tempdict['ticker'] = ticker
    tempdict['date'] = date
    tempdict['tweet'] = tweet
    
    try:
        tempTimeframe = dailyClose.loc[str(date - timedelta(days=10)) : str(date)]
        tempTimeframe = tempTimeframe.loc[tempTimeframe['ticker'] == ticker].tail(5)
        tempdict['5dp'] = tempTimeframe['adjclose'].iloc[0]
        tempdict['4dp'] = tempTimeframe['adjclose'].iloc[1]
        tempdict['3dp'] = tempTimeframe['adjclose'].iloc[2]
        tempdict['2dp'] = tempTimeframe['adjclose'].iloc[3]
        tempdict['1dp'] = tempTimeframe['adjclose'].iloc[4]
    except:
        tempdict['5dp'] = None
        tempdict['4dp'] = None
        tempdict['3dp'] = None
        tempdict['2dp'] = None
        tempdict['1dp'] = None
    
    try:
        twoWeeks = dailyClose.loc[str(date + timedelta(days=14)): str(date + timedelta(days=18))]
        twoWeeks = twoWeeks.loc[twoWeeks['ticker'] == ticker].head(1)
        tempdict['twoWeeksClose'] = twoWeeks['adjclose'].iloc[0]
    except:
        tempdict['twoWeeksClose'] = None
    
    try:
        threeMonths = dailyClose.loc[str(date + timedelta(days=90)): str(date + timedelta(days=93))]
        threeMonths = threeMonths.loc[threeMonths['ticker'] == ticker].head(1)
        tempdict['threeMonthsClose'] = threeMonths['adjclose'][0]
    except:
        tempdict['threeMonthsClose'] = None
    
    try:
        oneYear = dailyClose.loc[str(date + timedelta(days=365)): str(date + timedelta(days=368))]
        oneYear = oneYear.loc[oneYear['ticker'] == ticker].head(1)
        tempdict['oneYearClose'] = oneYear['adjclose'][0]
    except:
        tempdict['oneYearClose'] = None



    allData = allData.append(pd.DataFrame(pd.Series(tempdict)).T)